## Логистическая регрессия

[Ссылка на полную статью](https://habr.com/ru/company/ods/blog/323890/)

In [5]:
import pandas as pd
import numpy as np

In [6]:
sessions = pd.read_csv('../data/alice/train_sessions.csv', index_col='session_id')

In [7]:
sessions.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [8]:
sessions[['time%s' % i for i in range(1,11)]]

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
1,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-02-22 11:19:50,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15,2014-02-22 11:20:16
3,2013-12-16 16:40:17,2013-12-16 16:40:18,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22,2013-12-16 16:40:24
4,2014-03-28 10:52:12,2014-03-28 10:52:42,2014-03-28 10:53:12,2014-03-28 10:53:42,2014-03-28 10:54:12,2014-03-28 10:54:42,2014-03-28 10:55:12,2014-03-28 10:55:42,2014-03-28 10:56:12,2014-03-28 10:56:42
5,2014-02-28 10:53:05,2014-02-28 10:55:22,2014-02-28 10:55:22,2014-02-28 10:55:23,2014-02-28 10:55:23,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:57:06,2014-02-28 10:57:11
...,...,...,...,...,...,...,...,...,...,...
253557,2013-11-25 10:26:54,2013-11-25 10:26:58,2013-11-25 10:27:03,2013-11-25 10:27:04,2013-11-25 10:27:13,2013-11-25 10:27:16,2013-11-25 10:27:28,2013-11-25 10:27:40,2013-11-25 10:27:52,2013-11-25 10:27:53
253558,2013-03-12 16:01:15,2013-03-12 16:01:16,2013-03-12 16:01:16,2013-03-12 16:01:17,2013-03-12 16:01:17,2013-03-12 16:01:17,2013-03-12 16:01:18,2013-03-12 16:01:18,2013-03-12 16:01:18,2013-03-12 16:01:18
253559,2013-09-12 14:05:03,2013-09-12 14:05:10,2013-09-12 14:05:10,2013-09-12 14:06:29,2013-09-12 14:06:30,NaN,NaN,NaN,NaN,NaN


In [9]:
times = ['time%s' % i for i in range(1,11)]
sites = ['site%s' % i for i in range(1,11)]
sessions[times] = sessions[times].apply(pd.to_datetime)
sessions[sites] = sessions[sites].fillna(0).astype('int')
sessions.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,0,NaT,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
2,890,2014-02-22 11:19:50,941,2014-02-22 11:19:50,3847,2014-02-22 11:19:51,941,2014-02-22 11:19:51,942,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847,2014-02-22 11:19:52,3846,2014-02-22 11:19:52,1516,2014-02-22 11:20:15,1518,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39,2013-12-16 16:40:18,14768,2013-12-16 16:40:19,14769,2013-12-16 16:40:19,37,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768,2013-12-16 16:40:20,14768,2013-12-16 16:40:21,14768,2013-12-16 16:40:22,14768,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782,2014-03-28 10:52:42,782,2014-03-28 10:53:12,782,2014-03-28 10:53:42,782,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782,2014-03-28 10:55:12,782,2014-03-28 10:55:42,782,2014-03-28 10:56:12,782,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177,2014-02-28 10:55:22,175,2014-02-28 10:55:22,178,2014-02-28 10:55:23,177,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175,2014-02-28 10:55:59,177,2014-02-28 10:55:59,177,2014-02-28 10:57:06,178,2014-02-28 10:57:11,0


In [10]:
df_sites = sessions[sites]
df_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178


In [11]:
from scipy.sparse import csr_matrix

In [12]:
csr_matrix?

Init signature: csr_matrix(arg1, shape=None, dtype=None, copy=False)
Docstring:     
Compressed Sparse Row matrix

This can be instantiated in several ways:
    csr_matrix(D)
        with a dense matrix or rank-2 ndarray D

    csr_matrix(S)
        with another sparse matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
        is the standard CSR representation where the column indices for
        row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
        corresponding values are stored in ``data[indptr[i]:indptr[i+1]]``.
        If the shape parameter is not supplied, the matrix dimensions
        are inferred 

In [13]:
sites_flatten = df_sites.values.flatten()
sites_flatten.shape[0]

2535610

In [14]:
sites_flatten

array([718,   0,   0, ...,  30,  35,  33])

In [15]:
range(0,sites_flatten.shape[0]+10,10)

range(0, 2535620, 10)

In [16]:
sites_sparse = csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0,sites_flatten.shape[0]+10,10),
    ), dtype=np.int8
)[:,1:]

In [17]:
sites_sparse.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [18]:
sites_sparse.A.shape

(253561, 41601)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = sessions['target']

In [20]:
sites_sparse

<253561x41601 sparse matrix of type '<class 'numpy.int8'>'
	with 2412880 stored elements in Compressed Sparse Row format>

In [21]:
X_train, X_test, y_train, y_test = train_test_split(sites_sparse, y, random_state=10, test_size= 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(177492, 41601) (76069, 41601) (177492,) (76069,)


In [22]:
logit = LogisticRegression(random_state=10, max_iter=1000)
logit.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=10)

In [23]:
y_predict = logit.predict(X_test)

In [24]:
len(y_predict[y_predict==1])

166

In [25]:
y_test[y_test==1]

session_id
118363    1
253140    1
93981     1
68123     1
138364    1
         ..
160969    1
154528    1
40495     1
143544    1
211793    1
Name: target, Length: 675, dtype: int64

In [26]:
from sklearn import metrics

In [27]:
metrics.confusion_matrix(y_test, y_predict)

array([[75343,    51],
       [  560,   115]])

$$precision = \frac{TP}{TP + FP}\quad$$
$$recall = TPR = \frac{TP}{TP + FN}\quad$$ 

In [28]:
metrics.precision_score(y_test, y_predict)

0.6927710843373494

In [29]:
metrics.recall_score(y_test, y_predict)

0.17037037037037037

In [30]:
from sklearn.metrics import roc_auc_score, classification_report
metrics.precision_score(y_test, y_predict)

0.6927710843373494

In [31]:
def get_precision(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.3)
    logit = LogisticRegression(random_state=10, max_iter=1000)
    y_predict = logit.fit(X_train, y_train).predict(X_test)
    return metrics.precision_score(y_test, y_predict)

In [32]:
get_precision(sites_sparse, y)

0.6927710843373494

In [33]:
times

['time1',
 'time2',
 'time3',
 'time4',
 'time5',
 'time6',
 'time7',
 'time8',
 'time9',
 'time10']

In [34]:
first_time=sessions['time1']
first_time

session_id
1        2014-02-20 10:02:45
2        2014-02-22 11:19:50
3        2013-12-16 16:40:17
4        2014-03-28 10:52:12
5        2014-02-28 10:53:05
                 ...        
253557   2013-11-25 10:26:54
253558   2013-03-12 16:01:15
253559   2013-09-12 14:05:03
253560   2013-12-19 15:20:22
253561   2014-04-25 09:56:52
Name: time1, Length: 253561, dtype: datetime64[ns]

In [35]:
morning = first_time.apply(lambda x: 1 if x.hour < 12 else 0)

In [36]:
morning

session_id
1         1
2         1
3         0
4         1
5         1
         ..
253557    1
253558    0
253559    0
253560    0
253561    1
Name: time1, Length: 253561, dtype: int64

In [37]:
from scipy.sparse import hstack

In [38]:
sites_sparse

<253561x41601 sparse matrix of type '<class 'numpy.int8'>'
	with 2412880 stored elements in Compressed Sparse Row format>

In [39]:
morning.values[:,None]

array([[1],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [40]:
X = hstack([sites_sparse, morning.values.reshape(-1,1)])
get_precision(X,y)

0.7403314917127072

🛠 Добавить новые параметры. Добиться улучшения модели. 